In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from faiss import IndexFlatL2
import faiss
import pickle
from pathlib import Path
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
import torch
import os
import sys
from huggingface_hub import login

c:\Users\ASTU-PG\Intelligent-Complaint-Analysis-FinancialServices\.ienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sys.path.append(os.path.abspath('../src/'))

In [5]:
from loggers import get_logger
from rag_helper import rag_pipeline

In [6]:
logger = get_logger(__name__)

In [7]:
# Set up directories
VECTOR_STORE_DIR = Path('../vector_store')
DATA_DIR = Path('../data')

#### Load vector stor and metadata

In [8]:
# Load vector store and metadata
try:
    index = faiss.read_index(str(VECTOR_STORE_DIR / 'faiss_index.faiss'))
    with open(VECTOR_STORE_DIR / 'metadata.pkl', 'rb') as f:
        store_data = pickle.load(f)
        chunks = store_data['chunks']
        metadata = store_data['metadata']
    logger.info("Vector store and metadata loaded successfully.")
except FileNotFoundError as e:
    logger.error(f"Vector store files not found: {e}")
    raise
except Exception as e:
    logger.error(f"Error loading vector store or metadata: {e}")
    raise

2025-07-15 17:52:26,079 - INFO - Vector store and metadata loaded successfully.


#### Initialize Embedding Model

In [9]:
# Initialize embedding model with explicit checks
try:
    logger.info("Loading embedding model 'sentence-transformers/all-MiniLM-L6-v2'...")
    embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    if embedding_model is None:
        raise ValueError("Embedding model is None after initialization.")
    # Test the model
    test_embedding = embedding_model.encode(["test sentence"])
    logger.info(f"Embedding model loaded successfully. Test embedding shape: {test_embedding.shape}")
except Exception as e:
    logger.error(f"Failed to load or test embedding model 'sentence-transformers/all-MiniLM-L6-v2': {e}")
    logger.error("Ensure 'sentence-transformers' is installed and the model is accessible.")
    raise

2025-07-15 17:52:37,162 - INFO - Loading embedding model 'sentence-transformers/all-MiniLM-L6-v2'...
2025-07-15 17:52:52,060 - INFO - Embedding model loaded successfully. Test embedding shape: (1, 384)


#### Initiliaze th LLM

In [10]:
# Initialize LLM (try bart-base, fallback to flan-t5-base)
try:
        llm_pipeline = pipeline(
            "text2text-generation",
            model="google/flan-t5-base",
            device=0 if torch.cuda.is_available() else -1,
            max_length=200,
            do_sample=True,
            temperature=0.7
        )
        test_output = llm_pipeline("What are common issues with credit card billing disputes?")[0]['generated_text']
        logger.info(f"LLM (flan-t5-small) loaded and tested successfully. Test output: {test_output[:50]}...")
except Exception as e:
        logger.error(f"Failed to load flan-t5-small: {e}")
        logger.error("Ensure internet connection and try clearing cache: rm -rf ~/.cache/huggingface/hub")
        raise

llm = HuggingFacePipeline(pipeline=llm_pipeline)


Device set to use cpu


2025-07-15 17:53:53,992 - INFO - LLM (flan-t5-small) loaded and tested successfully. Test output: credit card fraud...


C:\Users\ASTU-PG\AppData\Local\Temp\ipykernel_6384\842926244.py:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [11]:
# Define prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a financial analyst assistant for CrediTrust. Your task is to answer questions about customer complaints based solely on the provided context. If the context doesn't contain enough information to answer the question, state clearly that you don't have sufficient information. Provide a concise and accurate response.

Context: {context}

Question: {question}

Answer:
"""
)



In [12]:
# Qualitative evaluation
evaluation_questions = [
    "What are common issues with credit card billing disputes?",
    "Why do customers complain about Buy Now, Pay Later services?",
    "Are there any complaints about unauthorized transactions in money transfers?",
    "What problems do people face with savings account fees?",
    "How do personal loan complaints differ from credit card complaints?"
]


In [13]:
evaluation_results = []
for question in evaluation_questions:
    try:
        result = rag_pipeline(question, llm, embedding_model, index, chunks, metadata, prompt_template)
        quality_score = 3
        comment = "Review answer and sources for accuracy and relevance."
        evaluation_results.append({
            'question': question,
            'answer': result['answer'][:200] + "..." if len(result['answer']) > 200 else result['answer'],
            'sources': [f"Chunk ID: {chunk['metadata']['chunk_id']}, Product: {chunk['metadata']['product']}, Text: {chunk['text'][:100]}..." for chunk in result['retrieved_chunks'][:2]],
            'quality_score': quality_score,
            'comment': comment
        })
    except Exception as e:
        logger.error(f"Error evaluating question '{question}': {e}")
        evaluation_results.append({
            'question': question,
            'answer': "Error generating answer.",
            'sources': [],
            'quality_score': 1,
            'comment': f"Failed due to: {str(e)}"
        })


2025-07-15 17:54:09,995 - INFO - Retrieved 5 chunks for query: What are common issues with credit card billing disputes?


c:\Users\ASTU-PG\Intelligent-Complaint-Analysis-FinancialServices\src\rag_helper.py:43: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


2025-07-15 17:54:14,597 - INFO - Generated answer for query: What are common issues with credit card billing disputes?
2025-07-15 17:54:14,802 - INFO - Retrieved 5 chunks for query: Why do customers complain about Buy Now, Pay Later services?
2025-07-15 17:54:39,575 - INFO - Generated answer for query: Why do customers complain about Buy Now, Pay Later services?
2025-07-15 17:54:39,665 - INFO - Retrieved 5 chunks for query: Are there any complaints about unauthorized transactions in money transfers?
2025-07-15 17:54:40,920 - INFO - Generated answer for query: Are there any complaints about unauthorized transactions in money transfers?
2025-07-15 17:54:41,030 - INFO - Retrieved 5 chunks for query: What problems do people face with savings account fees?
2025-07-15 17:54:48,730 - INFO - Generated answer for query: What problems do people face with savings account fees?
2025-07-15 17:54:48,814 - INFO - Retrieved 5 chunks for query: How do personal loan complaints differ from credit card co

In [14]:
# Save evaluation results
try:
    evaluation_df = pd.DataFrame(evaluation_results)
    evaluation_df.to_markdown(DATA_DIR / 'evaluation_table.md', index=False)
    logger.info(f"Evaluation table saved to: {DATA_DIR / 'evaluation_table.md'}")
except Exception as e:
    logger.error(f"Error saving evaluation table: {e}")
    raise


2025-07-15 17:54:56,311 - INFO - Evaluation table saved to: ..\data\evaluation_table.md


In [15]:
# Print sample evaluation
print("\nSample Evaluation Results:")
print(evaluation_df.to_markdown(index=False))



Sample Evaluation Results:
| question                                                                     | answer                                                                                                                                                                                                      | sources                                                                                                                                                                                                                                                                                                            |   quality_score | comment                                               |
|:-----------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------